## <center>Implement Neural Network</center>

In this project, I have used 'Insurance dataset' that will help us evaluate how likely an individual would buy an insurance based on Age and Affordability.
Also built Gradient Descent and Neural network from scratch by using plain python.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import keras
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
df= pd.read_csv(r'C:\Users\Administrator\machine learning\insurance_dataset.csv')
df.head(10)

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1
5,56,1,1
6,55,0,0
7,60,0,1
8,62,1,1
9,61,1,1


#### Scaled age column as need to have both affordibility and age columns on the same scale.

In [3]:
df_scale= df.copy()
df_scale.age= df_scale.age/100 # from 1 to 100
df_scale.head(10)

,age,affordibility,bought_insurance
0,0.22,1,0
1,0.25,0,0
2,0.47,1,1
3,0.52,0,0
4,0.46,1,1
5,0.56,1,1
6,0.55,0,0
7,0.60,0,1
8,0.62,1,1
9,0.61,1,1


In [4]:
# Splitting the dataset.

x_train,x_test,y_train,y_test= train_test_split(df_scale[['age','affordibility']], df_scale.bought_insurance, train_size=0.8, random_state=1 )

In [5]:
len(x_train)

22

In [6]:
len(x_test)

6

### Implemented Neural network using Tensorflow

In [7]:
from keras.layers import Dense, Activation
model= Sequential([
    Dense(1, input_shape=(2,), activation= 'sigmoid', kernel_initializer= 'ones', bias_initializer= 'zeros')
])

In [8]:
model.compile(loss= 'binary_crossentropy',
             optimizer= 'adam',
             metrics= ['accuracy'])

In [9]:
model.fit(x_train,y_train) # one epoch

Epoch 1/1
22/22 [==============================] - 0s 12ms/step - loss: 0.7068 - accuracy: 0.5000


In [10]:
weight, bias= model.get_weights()

In [11]:
weight    # starting weights of age, affordibility

array([[0.9992559],
       [0.9992559]], dtype=float32)

In [12]:
bias   # starting bias

array([-0.00074412], dtype=float32)

In [13]:
model.fit(x_train,y_train,epochs= 3500) # increased epochs

Epoch 1/3500
22/22 [==============================] - 0s 1000us/step - loss: 0.7065 - accuracy: 0.5000
Epoch 2/3500
22/22 [==============================] - 0s 91us/step - loss: 0.7062 - accuracy: 0.5000
Epoch 3/3500
22/22 [==============================] - 0s 136us/step - loss: 0.7059 - accuracy: 0.5000
Epoch 4/3500
22/22 [==============================] - 0s 136us/step - loss: 0.7055 - accuracy: 0.5000
Epoch 5/3500
22/22 [==============================] - 0s 91us/step - loss: 0.7052 - accuracy: 0.5000
Epoch 6/3500
22/22 [==============================] - 0s 136us/step - loss: 0.7048 - accuracy: 0.5000
Epoch 7/3500
22/22 [==============================] - 0s 136us/step - loss: 0.7044 - accuracy: 0.5000
Epoch 8/3500
22/22 [==============================] - 0s 136us/step - loss: 0.7041 - accuracy: 0.5000
Epoch 9/3500
22/22 [==============================] - 0s 273us/step - loss: 0.7037 - accuracy: 0.5000
Epoch 10/3500
22/22 [==============================] - 0s 136us/step - loss: 0.7033

In [14]:
model.evaluate(x_test, y_test)

6/6 [==============================] - 0s 38ms/step


[0.4519183933734894, 1.0]

In [15]:
# weights of age, affordibility and bias after 3500 epochs
weights, bias= model.get_weights() 
print(f'W1:{weights[0]}, W2:{weights[1]}, bias:{bias}')

W1:[3.861938], W2:[1.0980022], bias:[-2.1415591]


In [16]:
x_test

,age,affordibility
24,0.50,1
17,0.58,1
19,0.18,1
20,0.21,1
14,0.49,1
3,0.52,0


In [17]:
yp= model.predict(x_test)
yp

array([[0.7083558 ],
       [0.7678779 ],
       [0.41376856],
       [0.44212312],
       [0.7003138 ],
       [0.4667115 ]], dtype=float32)

In [18]:
y_test

24    1
17    1
19    0
20    0
14    1
3     0
Name: bought_insurance, dtype: int64

##### sigmoid function

In [19]:
from math import exp

def sigmoid(x):
    return 1/(1+exp(-x))

##### Instead of model.predict, wrote our own prediction function that uses w1,w2 and bias

In [20]:
def prediction_func(x1, x2):
    weighted_sum= weights[0]*x1 + weights[1]*x2 + bias  # Used weights from above model.
    return sigmoid(weighted_sum)

In [21]:
prediction_func(0.52,0)

0.4667114835204448

### Implemented Gradient Descent from scratch
The goal is to come up with same w1, w2 and bias that keras model calculated.

In [22]:
# sigoid func for arrays.

def sigmoid_np(x):
    return 1/(1+np.exp(-x))

In [23]:
sigmoid_np(np.array([42,-33,3]))

array([1.00000000e+00, 4.65888615e-15, 9.52574127e-01])

#### Implemented Log Loss or Binary Cross Entropy

In [24]:
def bce(y_true, y_predicted):
    epsilon= 1e-15
    error= 0  
    y_predicted_new= [ max(i, epsilon) for i in y_predicted]
    y_predicted_new= [ min(i, 1-epsilon) for i in y_predicted_new]  
    y_predicted_new= np.array(y_predicted_new)
    bce= -np.mean(y_true * np.log(y_predicted_new) +(1-y_true) * np.log(1-y_predicted_new))        
    return bce

In [25]:
def gradient_descent(x1,x2,y_true,epochs,loss_threshold):
    
    w1=w2=1
    bias=0
    learning_rate= 0.1
    n= len(x1)
    
    for i in range(epochs):
        weighted_sum= w1*x1 + w2*x2 + bias 
        y_predicted= sigmoid_np(weighted_sum)
        loss= bce(y_true, y_predicted)
    
        w1d= (1/n)*(np.dot(np.transpose(x1),(y_predicted-y_true)))
        w2d= (1/n)*(np.dot(np.transpose(x2),(y_predicted-y_true)))
        bias_d= np.mean(y_predicted-y_true)
    
        w1= w1-learning_rate*w1d
        w2= w2-learning_rate*w2d
        bias= bias-learning_rate*bias_d
        
        if loss <= loss_threshold: # as need to compare weights and bias with tensorflow model.
            break
    
        print(f'W1:{w1}, W2:{w2}, bias:{bias}, loss:{loss}')
    return w1, w2, bias

In [26]:
gradient_descent(x_train.age, x_train.affordibility, y_train, 2000, 0.4448)

W1:0.9955324184480593, W2:0.9896158041440797, bias:-0.022453413341607935, loss:0.7067871004847728
W1:0.9912814042232098, W2:0.9795794051967283, bias:-0.044348014675807935, loss:0.7005530386735819
W1:0.9872442107768984, W2:0.9698878053452621, bias:-0.06569161003700413, loss:0.6946532581994433
W1:0.9834178723446488, W2:0.9605375326271981, bias:-0.08649251207608602, loss:0.68907402868994
W1:0.9797992216041957, W2:0.9515246711861695, bias:-0.10675949473661692, loss:0.6838016738344731
W1:0.9763849076033503, W2:0.9428448927540957, bias:-0.1265017476649895, loss:0.6788226288002475
W1:0.9731714137777066, W2:0.9344934889653662, bias:-0.145728830775256, loss:0.6741234928228154
W1:0.9701550758915072, W2:0.9264654041310006, bias:-0.16445062935380161, loss:0.6696910767742831
W1:0.9673320997498964, W2:0.9187552681276246, bias:-0.1826773100501186, loss:0.665512445608952
W1:0.9646985785468727, W2:0.9113574290864684, bias:-0.20041927805891058, loss:0.6615749556744612
W1:0.9622505097300104, W2:0.9042659

(5.49774501685961, 1.2487858610697378, -2.954758567940789)

Tensorflow model= W1:[5.506266], W2:[1.213402], bias:[-2.9263778]

#### So the calculated weights and bias are as same as we have got from Tensorflow model!

### Custom Neural Network

In [93]:
class NeuralNet:
    def __init__(self):
        self.w1=1 
        self.w2=1 
        self.bias=0
    
    def fit(self,x_train,y_train,epochs,loss_threshold):
        self.w1, self.w2, self.bias= self.gradient_descent(x_train['age'],x_train['affordibility'],y_train,epochs,loss_threshold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self,x_test):
        weighted_sum= self.w1*x_test['age'] + self.w2*x_test['affordibility'] + self.bias
        return sigmoid_np(weighted_sum)
    
    def gradient_descent(self,x1,x2,y_true,epochs,loss_threshold):

        w1=w2=1
        bias=0
        learning_rate= 0.1
        n= len(x1)

        for i in range(epochs):
            weighted_sum= w1*x1 + w2*x2 + bias 
            y_predicted= sigmoid_np(weighted_sum)
            loss= bce(y_true, y_predicted)

            w1d= (1/n)*(np.dot(np.transpose(x1),(y_predicted-y_true)))
            w2d= (1/n)*(np.dot(np.transpose(x2),(y_predicted-y_true)))
            bias_d= np.mean(y_predicted-y_true)

            w1= w1-learning_rate*w1d
            w2= w2-learning_rate*w2d
            bias= bias-learning_rate*bias_d
            
            if i%50 == 0:
                print(f'W1:{w1}, W2:{w2}, bias:{bias}, loss:{loss}')
                
            if loss <= loss_threshold: # as need to compare weights and bias with tensorflow model.
                break
                
        return w1, w2, bias    

In [94]:
custom_NN= NeuralNet()
custom_NN.fit(x_train,y_train,epochs=2000,loss_threshold= 0.4448)

W1:0.9955324184480593, W2:0.9896158041440797, bias:-0.022453413341607935, loss:0.7067871004847728
W1:0.9708672870973896, W2:0.7981883015810478, bias:-0.6366458135088361, loss:0.602694555804264
W1:1.100741604360807, W2:0.8564166656074311, bias:-0.8499245195363202, loss:0.5888997018450574
W1:1.2539198429843401, W2:0.937337328883083, bias:-0.9894965460488974, loss:0.5789591309909329
W1:1.4094757519678573, W2:1.0076233252928215, bias:-1.1082266349423924, loss:0.5702902768111181
W1:1.5642090982796752, W2:1.0652009011674495, bias:-1.2152354697597914, loss:0.5625327058939591
W1:1.7174485674268813, W2:1.1118622922549926, bias:-1.3132207424789202, loss:0.5554686354459272
W1:1.868924432229138, W2:1.1495354204091275, bias:-1.4037127174888344, loss:0.548948811805506
W1:2.018462447217224, W2:1.1798467673178745, bias:-1.4878634021735015, loss:0.5428691129228838
W1:2.165933161889934, W2:1.2041253464309152, bias:-1.5666020015852051, loss:0.5371557121475228
W1:2.3112389680026806, W2:1.2234546790050531,

Tensorflow model= W1:[5.506266], W2:[1.213402], bias:[-2.9263778]
##### This shows that in the end we were able to come up with somewhat same value of w1,w2 and bias using a plain python implementation of neural network.

In [95]:
x_test

,age,affordibility
24,0.50,1
17,0.58,1
19,0.18,1
20,0.21,1
14,0.49,1
3,0.52,0


##### Prediction using custom NN.

In [96]:
custom_NN.predict(x_test)

24    0.739409
17    0.814983
19    0.328191
20    0.365529
14    0.728677
3     0.476036
dtype: float64

##### Prediction using Tensorflow model.

In [97]:
model.predict(x_test)

array([[0.7083558 ],
       [0.7678779 ],
       [0.41376856],
       [0.44212312],
       [0.7003138 ],
       [0.4667115 ]], dtype=float32)

In [98]:
y_test

24    1
17    1
19    0
20    0
14    1
3     0
Name: bought_insurance, dtype: int64